In [ ]:
from PIL import Image
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask.distributed import Client, progress
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

In [ ]:
def sparseH(L, J, h, periodic):
    
    """
    generates the sparse Hamiltonian matrix for the quantum Ising chain
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (csr_matrix): sparse matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    # initialize 
    H_data = []
    H_rows = []
    H_cols = []
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1, L + 1): # iterate over all sites
            
            alpha = beta ^ (1 << (j - 1)) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            "Keep track of the indices with non-zero matrix elements"
            
            H_data.append(-h)
            H_rows.append(alpha)
            H_cols.append(beta)
    
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim):  # iterate over all states
        
        A = 0
        
        for j in range(1, L): # iterate over all sites
            
            if 2 * (alpha & (1 << (j - 1))) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
                
        "Handling periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))): # Check if the states at either end have the same spin
                
                A -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                A += J # if not, increase the energy by the ising interaction term
        
        if A != 0: # Check if the resulting matrix element is non-zero, if so, keep track of it
        
            H_data.append(A)
            H_rows.append(alpha)
            H_cols.append(alpha)

    H_data = np.array(H_data, dtype=float) # convert the list into a np array
    
    H = scipy.sparse.csr_matrix((H_data, (H_rows, H_cols)), shape=(dim, dim), dtype=np.float64) # make it into a csr sparse matrix
    
    return H

In [ ]:
def entanglement_entropy(l, L, vec):
    psi_matrix = vec.reshape((2**l, 2**(L - l)))
    u, s, vh = np.linalg.svd(psi_matrix, compute_uv=True)
    lambdas = s**2
    S = -np.sum(lambdas * np.log(lambdas, out=np.zeros_like(lambdas), where=(lambdas != 0)))
    return S


In [ ]:
def compute_middle_spectrum_entropies(L_values, J, h, periodic, num_states):
    results = {}
    for L in L_values:
        H = sparseH(L, J, h, periodic).toarray()
        eigs, vecs = np.linalg.eigh(H)
        middle_indices = np.argsort(np.abs(eigs))[:num_states]
        non_deg_indices = np.unique(middle_indices)

        Entropies = []
        for l in tqdm(range(1, L), desc=f'Computing for L={L}'):
            avg_entropy = np.mean([entanglement_entropy(l, L, vecs[:, idx]) for idx in non_deg_indices])
            Entropies.append(avg_entropy)
        results[L] = Entropies
    return results

def plot_results(results):
    plt.figure(figsize=(10, 6))
    for L, entropies in results.items():
        plt.plot(np.arange(1, L)/L, entropies, label=f'L={L}')
    plt.xlabel(r'Normalized Subsystem Size $\frac{l}{L}$')
    plt.ylabel('Entanglement Entropy')
    plt.title('Entanglement Entropy vs. Subsystem Size Middle of the Spectrum\n Open B.C. paramagnetic phase')
    plt.legend()
    plt.grid(True)

In [ ]:
L_values = range(5, 14)
J = 1
h = 1.7
periodic = False
num_states = 5
entropies = compute_middle_spectrum_entropies(L_values, J, h, periodic, num_states)

plot_results(entropies)
plt.savefig(os.path.join(directory, 'openmiddlepara.png'), dpi = 400)
plt.show()

In [ ]:
def compute_avg_entropy(L_values, J, h, periodic, num_states):
    avg_entropies = []
    for L in tqdm(L_values, desc='Calculating Entanglement Entropy'):
        H = sparseH(L, J, h, periodic).toarray()
        eigs, vecs = np.linalg.eigh(H)
        middle_indices = np.argsort(np.abs(eigs))[:num_states]
        non_deg_indices = np.unique(middle_indices)

        entropies = [entanglement_entropy(L // 2, L, vecs[:, idx]) for idx in non_deg_indices]
        avg_entropy = np.mean(entropies)
        avg_entropies.append(avg_entropy)

    return avg_entropies

L_values = range(2, 14)
J = 1
h = 1.7
periodic = False
num_states = 5

avg_entropies = compute_avg_entropy(L_values, J, h, periodic, num_states)

plt.plot(L_values, avg_entropies, label='open')
plt.xlabel('L')
plt.ylabel(r'S($L/2$; $L$)')
plt.title(r'S($L/2$; $L$) vs $L$ in the Paramagnetic Phase')
plt.legend()
plt.savefig(os.path.join(directory, 'SL2middle.png'), dpi = 400)
plt.show()

In [ ]:
def random_hilbert_vector(L):
    dim = 2 ** L
    vector = np.random.normal(size=(dim,)) + 1j * np.random.normal(size=(dim,))
    vector /= np.linalg.norm(vector)
    return vector

def plot_random_vector_entropy(L):
    vec = random_hilbert_vector(L)
    entropies = []
    for l in range(1, L):
        entropy = entanglement_entropy(l, L, vec)
        entropies.append(entropy)

    plt.plot(np.arange(1, L) / L, entropies, label=f'L={L}')
    plt.xlabel(r'Normalized Subsystem Size $\frac{l}{L}$')
    plt.ylabel('Entanglement Entropy')
    plt.title(f'Entanglement Entropy S($l$; $L$) for Random Vector')
    plt.grid(True)
    plt.legend()

Ls = range(5,14)
for L in Ls:
    plot_random_vector_entropy(L)
plt.savefig(os.path.join(directory, 'randomentropy.png'), dpi = 400)
plt.show()
